## **bronze-companies-information**
<p>This notebook collects and stores basic company metadata from Yahoo Finance, such as sector, industry, market cap, and location. It acts as a reference layer for enriching financial and news datasets with company-level context.</p>

#### **Importing necessary libraries**

In [1]:
import yfinance as yf
import pandas as pd
import json
import datetime

StatementMeta(, e2bd94cc-399a-4904-941f-3bdc3a973873, 5, Finished, Available, Finished)

#### **Variables**

In [ ]:
file_name = "comp_info_data"
layer_name = "bronze"

#### **Step 1: Load the company codes (tickers) from the CSV**

In [2]:
codes = spark.read.csv('Files/master_files/company_code.csv', header=True)
short_codes = [
    row['short_code'].strip()
    for row in codes.select('short_code').distinct().collect()
    if row['short_code']
]

StatementMeta(, e2bd94cc-399a-4904-941f-3bdc3a973873, 6, Finished, Available, Finished)

#### **Step 2: Fetch Companies Information**

In [3]:
output = []

for i in short_codes:
    try:
        tickers = yf.Ticker(i)
        info = tickers.info

        info_flat = {k: v for k, v in info.items() if not isinstance(v, (dict, list))}

        df = pd.DataFrame([info_flat])
        df['Company'] = i

        output.append(df)

    except Exception as e:
        print(f"An error occurred for {i}: {e}")


StatementMeta(, e2bd94cc-399a-4904-941f-3bdc3a973873, 7, Finished, Available, Finished)

HTTP Error 404: 
HTTP Error 404: 
HTTP Error 404: 


#### **Step 3: Combine all DataFrames and save for Bronze layer**

In [7]:
if output:
    final_com_info_df = pd.concat(output, ignore_index=True).astype(str)
    spark_df = spark.createDataFrame(final_com_info_df)

    spark_df.write.mode("overwrite").parquet(f"Files/{layer_name}/{file_name}")

    print(f"[SUCCESS] Stock info data written to Lakehouse")
else:
    print("[WARNING] No data to write.")

StatementMeta(, e2bd94cc-399a-4904-941f-3bdc3a973873, 11, Finished, Available, Finished)

/tmp/ipykernel_21801/155157834.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_com_info_df = pd.concat(output, ignore_index=True).astype(str)


[SUCCESS] Stock info data written to Lakehouse path: Files/bronze/comp_info_data
